In [26]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tqdm import trange

In [27]:
# 加载鸢尾花数据集
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df['label'] = iris.target
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label']

# 加载鸢尾花数据集
x = iris.data
y = iris.target

# 加载鸢是三分类，将它转为二分类
y = np.array([1 if i == 1 else 0 for i in y])
# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

x_train,y_train

(array([[4.6, 3.4, 1.4, 0.3],
        [5. , 3. , 1.6, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [5.8, 2.6, 4. , 1.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.1, 3.3, 1.7, 0.5],
        [5. , 3.2, 1.2, 0.2],
        [6.5, 2.8, 4.6, 1.5],
        [7.9, 3.8, 6.4, 2. ],
        [6.1, 3. , 4.9, 1.8],
        [5.4, 3. , 4.5, 1.5],
        [6.4, 2.7, 5.3, 1.9],
        [5.7, 2.9, 4.2, 1.3],
        [7.7, 3.8, 6.7, 2.2],
        [6.5, 3.2, 5.1, 2. ],
        [5.8, 2.7, 3.9, 1.2],
        [4.6, 3.6, 1. , 0.2],
        [6.9, 3.1, 5.4, 2.1],
        [6.7, 3.3, 5.7, 2.1],
        [6.3, 2.8, 5.1, 1.5],
        [5.5, 4.2, 1.4, 0.2],
        [4.4, 3.2, 1.3, 0.2],
        [5.8, 2.7, 5.1, 1.9],
        [5.4, 3.9, 1.7, 0.4],
        [5.5, 3.5, 1.3, 0.2],
        [5. , 3.5, 1.6, 0.6],
        [6.9, 3.1, 4.9, 1.5],
        [6.5, 3. , 5.8, 2.2],
        [6.7, 3.3, 5.7, 2.5],
        [6.1, 2.6, 5.6, 1.4],
        [5.4, 3.7, 1.5, 0.2],
        [6. , 3.4, 4.5, 1.6],
        [5.9, 3.2, 4.8, 1.8],
        [4

In [28]:
class Model:
    def __init__(self, train_data, train_label, num):
        self.train_data = train_data  # 训练集数据
        self.train_label = train_label  # 训练集标记
        self.num = num  # 多少个基本分类器
        # m为样本容量，n为特征数量
        m, n = np.shape(train_data)
        self.m = m  # m为样本容量
        self.n = n  # n为特征数量
        self.w = np.full(m, fill_value=1 / m)
        self.tree_arr = []
        self.alpha_arr = []

    def estimate(self, feature, divide, rule):
        if rule == 'H1L-1':
            H = 1
            L = -1
        else:
            H = -1
            L = 1

        error = 0
        predict_arr = []

        num = len(feature)

        # 遍历每个样本，判断样本的该特征值是否大于divide
        # 并预测相应的类
        for i in range(num):
            if feature[i] > divide:
                predict = H
            else:
                predict = L

            predict_arr.append(predict)

            # 如果预测值与标记不符
            # 则error + 该分类错误样本所对应的权值
            if predict != self.train_label[i]:
                error += self.w[i]

        return predict_arr, error

    def create_single_boosting_tree(self):
        # 错误的样本数
        error = 1
        tree_dict = {}
        for i in range(self.n):
            feature = self.train_data[:, i]

            for divide in [-0.5, 0.5, 1.5]:
                for rule in ['H1L-1', 'H-1L1']:
                    predict_arr, e = self.estimate(feature, divide, rule)

                    if e < error:
                        error = e
                    tree_dict['feature'] = i
                    tree_dict['divide'] = divide
                    tree_dict['rule'] = rule
                    tree_dict['error'] = e
                    tree_dict['PredictArr'] = predict_arr
        return tree_dict

    def create_boosting_tree(self):
        for N in trange(self.num):
            tree_dict = self.create_single_boosting_tree()
            self.tree_arr.append(tree_dict)
            
            e = tree_dict["error"]
            alpha = np.log((1 - e) / e) / 2
            self.alpha_arr.append(alpha)
            
            gxi=tree_dict["PredictArr"]
    
            exp = np.exp(-1*alpha*self.train_label*gxi)
            
            Z = np.dot(self.w, exp)
            
            self.w = self.w * exp / Z
            
    
    def fit(self):
        self.create_boosting_tree()

    def predict(self, feature, divide, rule, x):
        # 用于之后预测在该情况下每个样本属于哪个类
        if rule == 'H1L-1':
            H = 1
            L = -1
        else:
            H = -1
            L = 1
    
        # 根据特征值的大小返回相应预测值
        if x[feature] > divide:
            return H
        else:
            return L
    def test(self, test_data, test_label):
        error = 0
        for i in trange(len(test_label)):
            xi = test_data[i]
            yi = test_label[i]
            
            result = 0
            
            for j in range(self.num):
                tree = self.tree_arr[j]

                feature = tree['feature']
                divide = tree['divide']
                rule = tree['rule']

                weak_result = self.predict(feature, divide, rule, xi)

                alpha = self.alpha_arr[j]
                result += alpha * weak_result

            final_result = np.sign(result)

            # 如果分类错误，errorCnt + 1
            if final_result != yi:
                error += 1
        
        acc = 1- error / len(test_label)

        return acc


In [30]:
adr = Model(x_train, y_train, 100)
adr.fit()

acc = adr.test(x_test, y_test)
print("acc", acc)

100%|██████████| 45/45 [00:00<00:00, 15363.75it/s]

acc 0.022222222222222254
